## Set-up

Imports:

In [1]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '2'
os.environ['MKL_NUM_THREADS'] = '2'
os.environ["ADAPTIVE_PREFERENCE_ENV"] = "dev"

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt
from tqdm import trange
from scipy.optimize import minimize
import re


In [2]:

# Step 1: Load the Z data
data_path = os.path.expanduser("~/ryandew_adaptive_preference/RA/backend/data/scaled_vgg_pca10_dresses.csv")
raw_data = pd.read_csv(data_path)

def create_pairs(dresses):
    dresses_cols = [col for col in dresses.columns if re.compile(r"E\d+").match(col)]  # Columns like E1, E2, ...
    indexed_dresses = dresses.reset_index(drop=False)  # dresses dataframe with indices ranging from 0 to N
    # Artificial column for catesian product. Pandas version is too low to use how="cross"
    indexed_dresses["_merge_key"] = 0
    merged_all_dresses = pd.merge(indexed_dresses, indexed_dresses, on="_merge_key", how="inner", suffixes=("_1", "_2"))
    merged_filtered_dresses = merged_all_dresses[merged_all_dresses["index_1"] < merged_all_dresses["index_2"]]
    for ix, col in enumerate(dresses_cols, 1):
        merged_filtered_dresses[f"Diff{ix}"] = merged_filtered_dresses[f"{col}_1"] - merged_filtered_dresses[f"{col}_2"]
    return merged_filtered_dresses[
        ["Item_ID_1", "Item_ID_2", "URL_1", "URL_2"] + [f"Diff{ix}" for ix, _ in enumerate(dresses_cols, 1)]
    ].reset_index(drop=True)

all_items = create_pairs(raw_data)
embedding_cols_start = 'Diff1'
z_all = all_items.loc[:, embedding_cols_start:].values


# Step 2: Import the PrefOptim object:
os.chdir(os.path.expanduser("~/ryandew_adaptive_preference/RA/results_analysis/reoptimize/"))
from og_reopt import PrefOptim

# Step 3: Import useful analysis functions:
os.chdir(os.path.expanduser("~/ryandew_adaptive_preference/RA/results_analysis/reoptimize/"))
from utils import *


os.chdir(os.path.expanduser("~/ryandew_adaptive_preference/RA/results_analysis/reoptimize/"))
og_train = pd.read_csv(os.path.expanduser('~/Dropbox/1_proj/urbn/results/binary_mpc_results/analysis-7.0-knn1-binary-mpc-None/train_df.csv'))
og_test = pd.read_csv(os.path.expanduser('~/Dropbox/1_proj/urbn/results/binary_mpc_results/analysis-7.0-knn1-binary-mpc-None/test_df.csv'))

Useful functions:

In [3]:
# Extract training
def extract_train(i):
    train_i = og_train.iloc[i]
    z_shown = z_all[np_str(train_i.items_shown)]
    # Note to self: ratings is the original ratings, no negation
    ratings = np_str(train_i.ratings)
    return z_shown, ratings

def extract_test(i):
    # Extract test
    test_i = og_test.iloc[i]
    random_test_i = eval(test_i['random_test_predictions_ratings'])
    test_items = list(random_test_i.keys())
    old_preds = np.array([a for (a,b) in list(random_test_i.values())])
    test_ratings = np.array([b for (a,b) in list(random_test_i.values())])
    return test_items, test_ratings, old_preds

def extract_sym(i):
    z_shown, ratings = extract_train(i)
    # Compute symmetric z/ratings
    sym_z = np.vstack([z_shown, -z_shown])
    sym_ratings = np.append(ratings, -ratings)
    return sym_z, sym_ratings

def fit_comparison_dict(test_ratings, pred_ratings1, pred_ratings2, index=0):
    return {
        'old_cor': cor(test_ratings, pred_ratings1),
        'new_cor': cor(test_ratings, pred_ratings2),
        'old_sign_acc': sign_acc(test_ratings, pred_ratings1),
        'new_sign_acc': sign_acc(test_ratings, pred_ratings2),
        'old_rmse': rmse(test_ratings, pred_ratings1),
        'new_rmse': rmse(test_ratings, pred_ratings2),
    }

def hyper_comparison_dict(model1, model2, index=0):
    return {
        'old_noise': model1.hypers['noise'],
        'old_amp': model1.hypers['amp'],
        'old_ls': model1.hypers['ls'],
        'new_noise': model2.hypers['noise'],
        'new_amp': model2.hypers['amp'],
        'new_ls': model2.hypers['ls'],
    }


## Optimal-aggregate-hypers

### Optimal-using-RMSE

In [4]:
def average_rmse(hypers_list, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    # Compute average rmse across all people
    tot_rmse = 0
    N = len(z_train_list)
    for i in range(N):
        prefs = PrefOptim(
            z_shown = z_train_list[i], 
            ratings = ratings_train_list[i]
        )
        prefs.hypers = {'noise': hypers_list[0], 'amp': hypers_list[1], 'ls': hypers_list[2]}
        pred = prefs.compute_utility(z_test_list[i])
        tot_rmse += rmse(pred, ratings_test_list[i])
    return tot_rmse / N

def optimize_average_rmse(pars0, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    bnds = ((1e-6, 100), (1e-6, 100), (1e-6, 100))
    opt_out = minimize(
        average_rmse, 
        pars0, 
        args=(z_train_list, ratings_train_list, z_test_list, ratings_test_list), 
        method="L-BFGS-B", 
        bounds=bnds
    )
    return {"noise": opt_out.x[0], "amp": opt_out.x[1], "ls": opt_out.x[2]}

# Compute optimal aggregate RMSE:
z_train_list = []
ratings_train_list = []
z_test_list = []
ratings_test_list = []
for i in range(og_train.shape[0]):
    z_sym, ratings_sym = extract_sym(i)
    test_items, test_ratings, old_preds = extract_test(i)
    z_train_list.append(z_sym)
    ratings_train_list.append(ratings_sym)
    z_test_list.append(z_all[test_items])
    ratings_test_list.append(test_ratings)


In [5]:
# agg_hypers = optimize_average_rmse(
#     pars0=[0.1,1,0.5], 
#     z_train_list=z_train_list, 
#     ratings_train_list=ratings_train_list, 
#     z_test_list=z_test_list, 
#     ratings_test_list=ratings_test_list
# )

In [6]:
agg_hypers = {'noise': 6.88756714098321, 'amp': 7.148767946480032, 'ls': 5.357894316288602}

In [7]:
average_rmse(
    [agg_hypers['noise'], agg_hypers['amp'], agg_hypers['ls']], 
    z_train_list, 
    ratings_train_list, 
    z_test_list, 
    ratings_test_list
)

0.4897990867574739

In [8]:
def pred(i, hypers=None):
    # Symmetric training
    sym_z, sym_ratings = extract_sym(i)

    # Make predictions under new model + sym settings
    new_prefs = PrefOptim(z_shown = sym_z, ratings = sym_ratings)
    if hypers is not None:
        new_prefs.hypers = hypers
    return new_prefs.compute_utility(z_all[test_items])


In [9]:

pred(5, agg_hypers)

array([-0.00765583,  0.09131709,  0.11682866, -0.11992919, -0.03460138,
        0.06080502,  0.05859709, -0.02329211,  0.13875908,  0.27389191])

### Optimal-using-acc

In [10]:
def neg_average_acc(hypers_list, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    tot_acc = 0
    N = len(z_train_list)
    for i in range(N):
        prefs = PrefOptim(
            z_shown = z_train_list[i], 
            ratings = ratings_train_list[i]
        )
        prefs.hypers = {'noise': hypers_list[0], 'amp': hypers_list[1], 'ls': hypers_list[2]}
        pred = prefs.compute_utility(z_test_list[i])
        tot_acc += sign_acc(pred, ratings_test_list[i])
    return -tot_acc / N

def optimize_average_acc(pars0, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    bnds = ((1e-6, 100), (1e-6, 100), (1e-6, 100))
    opt_out = minimize(
        neg_average_acc, 
        pars0, 
        args=(z_train_list, ratings_train_list, z_test_list, ratings_test_list), 
        method="L-BFGS-B", 
        bounds=bnds
    )
    return {"noise": opt_out.x[0], "amp": opt_out.x[1], "ls": opt_out.x[2]}

# Compute optimal aggregate RMSE:
z_train_list = []
ratings_train_list = []
z_test_list = []
ratings_test_list = []
for i in range(og_train.shape[0]):
    z_sym, ratings_sym = extract_sym(i)
    test_items, test_ratings, old_preds = extract_test(i)
    z_train_list.append(z_sym)
    ratings_train_list.append(ratings_sym)
    z_test_list.append(z_all[test_items])
    ratings_test_list.append(test_ratings)


In [14]:
agg_hypers_acc = optimize_average_acc(
    pars0=[0.1,0.1,0.1], 
    z_train_list=z_train_list, 
    ratings_train_list=ratings_train_list, 
    z_test_list=z_test_list, 
    ratings_test_list=ratings_test_list
)

In [15]:
agg_hypers_acc

{'noise': 0.1, 'amp': 0.1, 'ls': 0.5}

In [16]:
-neg_average_acc([0.0001,1e-10,1e10], z_train_list, ratings_train_list, z_test_list, ratings_test_list)

0.4952702702702704

In [127]:
pred(0, {'noise': 0.0001, 'amp': 0.001, 'ls': 20000})

array([ 2.35708513e-09,  2.05170363e-08, -7.78199440e-09, -6.46654144e-10,
       -4.05372274e-09, -1.17180751e-08,  6.33636235e-09,  8.83542258e-09,
        2.05534828e-09, -3.69887803e-08])

In [120]:
nir_acc = 0
N = len(z_train_list)
for i in range(N):
    nir_acc += sign_acc(0.5, ratings_test_list[i])
nir_acc / N

0.5277027027027028

In [123]:
sign_acc(0.5, ratings_test_list[i])

0.7

So it seems that accuracy doesn't really need good hyperparameters to perform well...?

### Optimal-using-cor

In [17]:
def neg_average_cor(hypers_list, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    tot = 0
    N = len(z_train_list)
    for i in range(N):
        prefs = PrefOptim(
            z_shown = z_train_list[i], 
            ratings = ratings_train_list[i]
        )
        prefs.hypers = {'noise': hypers_list[0], 'amp': hypers_list[1], 'ls': hypers_list[2]}
        pred = prefs.compute_utility(z_test_list[i])
        cor_i = cor(pred, ratings_test_list[i])
        if np.isnan(cor_i) or np.isinf(cor_i):
            cor_i = 0
        tot += cor_i
    return -tot / N

def optimize_average_cor(pars0, z_train_list, ratings_train_list, z_test_list, ratings_test_list):
    bnds = ((1e-6, 100), (1e-6, 100), (1e-6, 100))
    opt_out = minimize(
        neg_average_cor, 
        pars0, 
        args=(z_train_list, ratings_train_list, z_test_list, ratings_test_list), 
        method="L-BFGS-B", 
        bounds=bnds
    )
    return {"noise": opt_out.x[0], "amp": opt_out.x[1], "ls": opt_out.x[2]}

# Compute optimal aggregate RMSE:
z_train_list = []
ratings_train_list = []
z_test_list = []
ratings_test_list = []
for i in range(og_train.shape[0]):
    z_sym, ratings_sym = extract_sym(i)
    test_items, test_ratings, old_preds = extract_test(i)
    z_train_list.append(z_sym)
    ratings_train_list.append(ratings_sym)
    z_test_list.append(z_all[test_items])
    ratings_test_list.append(test_ratings)


In [18]:
agg_hypers_cor = optimize_average_cor(
    pars0=[0.1,1,0.5], 
    z_train_list=z_train_list, 
    ratings_train_list=ratings_train_list, 
    z_test_list=z_test_list, 
    ratings_test_list=ratings_test_list
)

/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/ryandew/miniconda3/envs/ad

In [19]:
agg_hypers_cor

{'noise': 4.105707235134857, 'amp': 2.594658008010263, 'ls': 1.158389347265231}

In [20]:
-neg_average_cor(list(agg_hypers_cor.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list)

/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


0.17922362404167494

In [138]:
-neg_average_cor([0.1,1,0.5], z_train_list, ratings_train_list, z_test_list, ratings_test_list)

/Users/ryandew/miniconda3/envs/adp-analyze/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


0.14764889660231312

In [21]:
print("rmse optimized cor: ", -neg_average_cor(list(agg_hypers.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))
print("cor optimized cor: ", -neg_average_cor(list(agg_hypers_cor.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))
print("rmse optimized rmse: ", average_rmse(list(agg_hypers.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))
print("cor optimized rmse: ", average_rmse(list(agg_hypers_cor.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))
print("rmse optimized acc: ", -neg_average_acc(list(agg_hypers.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))
print("cor optimized acc: ", -neg_average_acc(list(agg_hypers_cor.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list))

/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


rmse optimized cor:  0.17688684812100539


/home/ryandew/miniconda3/envs/adpref/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cor optimized cor:  0.17922362404167494
rmse optimized rmse:  0.4897990867574739
cor optimized rmse:  0.49263141083193346
rmse optimized acc:  0.5777027027027029
cor optimized acc:  0.5797297297297302


In [28]:
# What about the old agg hypers?
old_hypers = {'noise': 0.38621058403938996,
 'amp': 1.905899429488762,
 'ls': 3.577134415736614}

In [29]:
-neg_average_acc(list(old_hypers.values()), z_train_list, ratings_train_list, z_test_list, ratings_test_list)

0.5581081081081083

In [22]:
agg_hypers_cor

{'noise': 4.105707235134857, 'amp': 2.594658008010263, 'ls': 1.158389347265231}

In [23]:
agg_hypers

{'noise': 6.88756714098321, 'amp': 7.148767946480032, 'ls': 5.357894316288602}

In [26]:
pred(i, hypers = agg_hypers_cor)

array([-0.23958414, -0.21016397, -0.22159125,  0.05438086, -0.06152196,
        0.03963397, -0.0778076 ,  0.17049908, -0.11361434, -0.17351468])

In [ ]:
pred(i, hypers = agg_hypers_cor)